# Mangrove tiles

We need create vector tiles for the Mangrove restoration and typology data provided by the client [here](https://github.com/Vizzuality/mangrove-atlas/issues/331).  
The original link to the source data is [here](https://tnc.app.box.com/s/8abiem7o8lyd9x1pvpe2yn2mdibgpcqk).  
The source data is an arcgis GDB and the data is stored in gcs bucket.  
The steps this pipe will take are:  
Pre.- script to move data from their original location to the gcs bucket.  
1.- Download raw data.  
2.- Tranform raw data to geojson with (mapshaper, geopandas, ...)  
3.- Generate the mbtile file with tippecanoe  
4.- Upload the mbtile file to gcs and mapbox as a tileset.  
5.- Comunicate to Design and FE team the tileset is ready.  

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import fiona


## 1. Download raw data
For now, working on local. Awaiting GCS permissions

# 2. Transform to geoJSON

In [3]:
gdb_file ='../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration_20190411.gdb'
layer = gpd.read_file(gdb_file, driver='FileGDB', layer=1)
layer.head()

,Class,ID,Type,Country,Region,Max_Area_20_ha,Area_loss_ha,Area_loss_pct,Rest_Area_Loss,Rest_Area_Loss_pct,...,AGB,People,Fish_Score,Fish_Score_Inv,Prop_loss1,Total_2016,Shape_Length,Shape_Area,Loss_Driver,geometry
0,Delta,30028,Delta_30028,Madagascar,East and Southern Africa,16309.90,881.10,5,546.28200,3,...,33368.813410,4100,5.640000e+08,7.490000e+08,10,15428.795789,1.863918e+06,1.850339e+08,None,"MULTIPOLYGON (((4953272.062 -2264345.607, 4953..."
1,Delta,50000,Delta_50000,Malaysia,Southeast Asia,26443.76,623.85,2,557.09805,2,...,64521.662450,0,4.480000e+08,5.990000e+08,10,25690.633999,2.400825e+06,2.671475e+08,None,"MULTIPOLYGON (((13237272.764 595333.394, 13237..."
2,Delta,50001,Delta_50001,Indonesia,Southeast Asia,99528.85,3765.96,4,3483.51300,3,...,388074.656600,300,5.567000e+09,8.190000e+09,10,95362.286892,1.001024e+07,1.002791e+09,None,"MULTIPOLYGON (((13107771.090 420349.509, 13107..."
3,Delta,8735,Delta_8735,Honduras,North and Central America and the Caribbean,167.84,18.60,11,9.13260,5,...,789.999563,0,4.570000e+08,7.630000e+08,10,149.237108,6.041805e+04,1.826093e+06,None,"MULTIPOLYGON (((-9545201.058 1798187.105, -954..."
4,Delta,8736,Delta_8736,Brazil,South America,157354.23,9609.83,6,7822.40162,4,...,677196.074000,13400,1.500000e+07,0.000000e+00,10,146881.955184,2.592286e+07,1.575569e+09,None,"MULTIPOLYGON (((-5401790.660 -171081.462, -540..."


In [4]:
layer.crs

<Derived Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
layer = layer.to_crs('EPSG:4326')

In [6]:
layer.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [7]:
out_path = '../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.geojson'

layer.to_file(out_path, driver="GeoJSON") 

## 3. Generate the mbtile file with tippecanoe  


In [11]:
in_file = '../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.geojson'
out_file = '../../../data/Typology_and_Restoration_Potential/Data/MOW_Global_Mangrove_Restoration.mbtiles'
!tippecanoe -zg -o $out_file --drop-densest-as-needed --force $in_file

For layer 0, using name "MOW_Global_Mangrove_Restoration"
6285 features, 240638248 bytes of geometry, 975439 bytes of separate metadata, 755228 bytes of string pool
Choosing a maxzoom of -z2 for features about 104264 feet (31780 meters) apart
Choosing a maxzoom of -z11 for resolution of about 136 feet (41 meters) within features
  99.9%  11/949/970    


In [12]:
print('ALL DONE!')

ALL DONE!


### Optional: visualize tiles with mbview 

In [16]:
#!npm install -g @mapbox/mbview
!mbview $out_file

missing access token, try `export MAPBOX_ACCESS_TOKEN=...`


## 4. Upload the mbtile file to gcs and mapbox as a tileset.  

## 5. Comunicate to Design and FE team the tileset is ready.  